# Foundation of Computer Science - Final Project

Roll number: 846965

Name: Pranav

Surname: Kasela

Academic year: 2018/2019

Course: DATA SCIENCE

Faculty: DEPARTMENT OF INFORMATICS, SYSTEMS AND COMMUNICATION (DISCO)

Class: LM-91-Classe delle lauree magistrali in Tecniche e metodi per la societa dell'informazione

e-mail: p.kasela@campus.unimib.it

In [44]:
#Imported here all the libraries needed for this exam
import pandas as pd
import numpy as np
import re

In [45]:
#Uncomment to print all the DataFrame or Series if needed, we print all the columns
pd.set_option('display.max_columns', None)  # or any number instead of None
#pd.set_option('display.max_rows', None)  # or any number instead of None
#pd.set_option('display.max_colwidth', -1)  # or any number instead of -1

In [46]:
#I use it because I like seeing above the tables it's name.
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [47]:
#Read both datasets
googlestore = pd.read_csv("./datasets/googleplaystore.csv")
googlereviews = pd.read_csv("./datasets/googleplaystore_user_reviews.csv")

In [48]:
#The row 10472 is problematics
googlestore.loc[[10472]]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


Best choice for the row 10472 is the record removal

In [49]:
googlestore=googlestore.drop(10472)

The datasets structure:

In [50]:
display("googlestore.head()")

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [51]:
display("googlereviews.head()")

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


## Exercise 1
1. Convert the app sizes to a number

In [52]:
#Checking the format of the data
googlestore.Size.unique()

array(['19M', '14M', '8.7M', '25M', '2.8M', '5.6M', '29M', '33M', '3.1M',
       '28M', '12M', '20M', '21M', '37M', '2.7M', '5.5M', '17M', '39M',
       '31M', '4.2M', '7.0M', '23M', '6.0M', '6.1M', '4.6M', '9.2M',
       '5.2M', '11M', '24M', 'Varies with device', '9.4M', '15M', '10M',
       '1.2M', '26M', '8.0M', '7.9M', '56M', '57M', '35M', '54M', '201k',
       '3.6M', '5.7M', '8.6M', '2.4M', '27M', '2.5M', '16M', '3.4M',
       '8.9M', '3.9M', '2.9M', '38M', '32M', '5.4M', '18M', '1.1M',
       '2.2M', '4.5M', '9.8M', '52M', '9.0M', '6.7M', '30M', '2.6M',
       '7.1M', '3.7M', '22M', '7.4M', '6.4M', '3.2M', '8.2M', '9.9M',
       '4.9M', '9.5M', '5.0M', '5.9M', '13M', '73M', '6.8M', '3.5M',
       '4.0M', '2.3M', '7.2M', '2.1M', '42M', '7.3M', '9.1M', '55M',
       '23k', '6.5M', '1.5M', '7.5M', '51M', '41M', '48M', '8.5M', '46M',
       '8.3M', '4.3M', '4.7M', '3.3M', '40M', '7.8M', '8.8M', '6.6M',
       '5.1M', '61M', '66M', '79k', '8.4M', '118k', '44M', '695k', '1.6M',
     

In [53]:
size_to_num = re.compile('(?P<number>\d+\.{0,1}\d*)(?P<prefix>\w*)')
#In this function for kilo lowercase k for kilo and uppercase M and G for Mega and Giga. 
#We Assume no file with any other unit otherwise the unit will be ignored. 
def prefix_to_mult(unit):
    if unit == 'G':
        return 1000000000
    if unit == 'M':
        return 1000000
    if unit == 'k':
        return 1000
    return 1

#This function can understand the format of the data given to it using regex,
#If the given data is not in the expected format it will return 'Not a Number'
def ConvertSizeToByte(Size):
    searched = size_to_num.search(Size)
    if searched is None:
        return np.nan
    else:
        prefix = searched.group('prefix')
        mult = prefix_to_mult(prefix)
        result = float(searched.group('number'))
        return int(result*mult)

We chose to create a new column for the converted data

In [54]:
googlestore['SizeInBytes'] = googlestore['Size'].apply(ConvertSizeToByte)

In [55]:
display("googlestore[['App','Size','SizeInBytes']].tail()") #Use tail because it has also a NaN

,App,Size,SizeInBytes
10836,Sya9a Maroc - FR,53M,53000000.0
10837,Fr. Mike Schmitz Audio Teachings,3.6M,3600000.0
10838,Parkinson Exercices FR,9.5M,9500000.0
10839,The SCP Foundation DB fr nn5n,Varies with device,NaN
10840,iHoroscope - 2018 Daily Horoscope & Astrology,19M,19000000.0


## Exercise 2
2. Convert the number of installs to a number

In [56]:
#Checking the format of the data
googlestore.Installs.unique()

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '50+', '100+', '500+',
       '10+', '1+', '5+', '0+', '0'], dtype=object)

In [57]:
display("googlestore[googlestore.Installs=='0']")
#This is the only row with install not in the normal format (with the +)
#but it is not a problem since we will ignore the + while searching for the number

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes
9148,Command & Conquer: Rivals,FAMILY,NaN,0,Varies with device,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",Varies with device,Varies with device,NaN


In [58]:
install_to_num = re.compile('(?P<number>[\d,]+)') #the format is num,num,num,...,num

#We find using regex the number in the format 
def installToNumber(installs):
    found = install_to_num.search(installs)
    if(found):
        replacedComma = found.group('number').replace(',','') #replace the commas with nothing
        return int(replacedComma)
    else:
        return np.nan

In [59]:
googlestore['InstallNumber']=googlestore['Installs'].apply(installToNumber)

In [60]:
display("googlestore[['App','Installs','InstallNumber']].head()")

,App,Installs,InstallNumber
0,Photo Editor & Candy Camera & Grid & ScrapBook,"10,000+",10000
1,Coloring book moana,"500,000+",500000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...","5,000,000+",5000000
3,Sketch - Draw & Paint,"50,000,000+",50000000
4,Pixel Draw - Number Art Coloring Book,"100,000+",100000


## Exercise 3
3. Transform “Varies with device” into a missing value

In [64]:
#replace 'Varies with device' with NaN.
googlestore.replace('Varies with device', np.nan,inplace=True)
#inplace is needed so the replace is executed on the dataframe itself

In [65]:
googlestore.tail()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up,53000000.0,5000
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up,3600000.0,100
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up,9500000.0,1000
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",NaN,NaN,NaN,1000
10840,iHoroscope - 2018 Daily Horoscope & Astrology,LIFESTYLE,4.5,398307,19M,"10,000,000+",Free,0,Everyone,Lifestyle,"July 25, 2018",NaN,NaN,19000000.0,10000000


## Exercise 4 (STILL TO BE COMPLETED)
4. Convert Current Ver and Android Ver into dotted number (e.g. 4.0.3 or 4.2)

Due to the data format we transform the Android version in 2 different column a minimum version and a maximum version considering the 'and up' value as the latest version 8.1 of android today.

In [67]:
#The versions with the W in the end are the wearable version we will not consider the W
googlestore['Android Ver'].value_counts()

4.1 and up       2451
4.0.3 and up     1501
4.0 and up       1375
4.4 and up        980
2.3 and up        652
5.0 and up        601
4.2 and up        394
2.3.3 and up      281
2.2 and up        244
4.3 and up        243
3.0 and up        241
2.1 and up        134
1.6 and up        116
6.0 and up         60
7.0 and up         42
3.2 and up         36
2.0 and up         32
5.1 and up         24
1.5 and up         20
4.4W and up        12
3.1 and up         10
2.0.1 and up        7
8.0 and up          6
7.1 and up          3
4.0.3 - 7.1.1       2
5.0 - 8.0           2
1.0 and up          2
2.2 - 7.1.1         1
5.0 - 6.0           1
4.1 - 7.1.1         1
5.0 - 7.1.1         1
7.0 - 7.1.1         1
Name: Android Ver, dtype: int64

In [68]:
print('There are {0} missing data values in Android Ver out of {1} more than 12%'
      .format(googlestore[googlestore['Android Ver'].isnull()].shape[0], googlestore.shape[0]))

There are 1364 missing data values in Android Ver out of 10840 more than 12%


The following cell contains the major_Android_ver considered 8.1 as of today.

We define two function one to compute the maximum version and one the minimum version.

In [69]:
MinVer_toNum=re.compile('^(?P<MinVer>\d+\.\d+\.{0,1}\d*)')
MaxVer_toNum=re.compile('(?P<MaxVer>\d+\.\d+\.{0,1}\d*)$')

#To be changed if new version is released, will be used instead of 'and up'
major_Android_ver = 8.1

#We Decide not to trow away the missing version since we would we throwing away more that 12% of the data!
def MinAndroidVersion_ToNum(version):
    if pd.isnull(version):
        return None
    MinVer=MinVer_toNum.search(version)
    if MinVer is not None:
        return MinVer.group('MinVer')
    else:
        return None
    
def MaxAndroidVersion_ToNum(version):
    if pd.isnull(version):
        return None
    MaxVer=MaxVer_toNum.search(version)
    if MaxVer is not None:
        return MaxVer.group('MaxVer')
    else:
        return major_Android_ver

In [70]:
googlestore['Min_Android_Ver']=googlestore['Android Ver'].apply(MinAndroidVersion_ToNum)
googlestore['Max_Android_Ver']=googlestore['Android Ver'].apply(MaxAndroidVersion_ToNum)

In [72]:
display("googlestore[['App','Android Ver','Min_Android_Ver','Max_Android_Ver']].tail()")
#The varies with device becomes None

,App,Android Ver,Min_Android_Ver,Max_Android_Ver
10836,Sya9a Maroc - FR,4.1 and up,4.1,8.1
10837,Fr. Mike Schmitz Audio Teachings,4.1 and up,4.1,8.1
10838,Parkinson Exercices FR,2.2 and up,2.2,8.1
10839,The SCP Foundation DB fr nn5n,NaN,None,None
10840,iHoroscope - 2018 Daily Horoscope & Astrology,NaN,None,None


In [74]:
googlestore['Current Ver'].value_counts()

1.0                 809
1.1                 264
1.2                 178
2.0                 151
1.3                 145
1.0.0               136
1.0.1               119
1.4                  88
1.5                  81
1.0.2                80
1.6                  65
1.0.3                62
2.1                  61
3.0                  59
1.0.4                58
1.7                  53
2.0.0                51
1.0.5                50
1.1.0                49
1.1.1                48
1.0.6                47
1.2.1                46
1.2.0                43
1.8                  42
4.0                  40
1.9                  37
2.3.2                35
2.4                  34
1.0.7                34
1.0.9                33
                   ... 
10.0.10               1
1.1.13                1
3.5.7                 1
10.0.3148             1
3.53.2                1
1.9.1.013             1
5.10.6.2.8466393      1
1.5.1-GP              1
2.2.2926              1
2.0.833.18            1
1.2.100         

In [75]:
#Special Value
googlestore[googlestore['Current Ver'] == "3rd Release Aug 2016"]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber,Min_Android_Ver,Max_Android_Ver
10691,Pin-fo,FAMILY,4.6,19,1.6M,500+,Free,0,Everyone,Entertainment,"August 15, 2016",3rd Release Aug 2016,2.2 and up,1600000.0,500,2.2,8.1
